In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
data_dir = '/content/drive/MyDrive/vehicles.csv'

In [4]:
data = pd.read_csv( data_dir)


# Preprocessing

In [5]:
data

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [6]:
data.isna()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
2,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
4,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False
426876,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False
426877,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False
426878,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False


In [7]:
data.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [8]:
data.isna().mean()

id              0.000000
url             0.000000
region          0.000000
region_url      0.000000
price           0.000000
year            0.002823
manufacturer    0.041337
model           0.012362
condition       0.407852
cylinders       0.416225
fuel            0.007058
odometer        0.010307
title_status    0.019308
transmission    0.005988
VIN             0.377254
drive           0.305863
size            0.717675
type            0.217527
paint_color     0.305011
image_url       0.000159
description     0.000164
county          1.000000
state           0.000000
lat             0.015342
long            0.015342
posting_date    0.000159
dtype: float64

In [9]:
data_v1 = data.drop(data.columns[data.isna().mean() > 0.25], axis = 1)

data_v1

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,gas,32226.0,clean,other,sedan,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,gas,12029.0,clean,other,sedan,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,diesel,4174.0,clean,other,hatchback,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,gas,30112.0,clean,other,sedan,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [10]:
unneeded_columns = ['id', 'url', 'region_url', 'image_url', 'description', 'posting_date', 'model']

data_v2 = data_v1.drop(unneeded_columns, axis = 1)

data_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [11]:
{column: len(data[column].unique()) for column in data_v2.columns if data_v2.dtypes[column] == 'object'}

{'fuel': 6,
 'manufacturer': 43,
 'region': 404,
 'state': 51,
 'title_status': 7,
 'transmission': 4,
 'type': 14}

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

In [13]:
impute = SimpleImputer(strategy = 'mean')

In [14]:
data_v2_num = data_v2.drop(['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state'], axis = 1)

In [15]:
impute.fit(data_v2_num)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [16]:
X = impute.transform(data_v2_num)

pd_d = pd.DataFrame(X, columns=data_v2_num.columns)

In [17]:
encoder = OneHotEncoder()

In [18]:
data_v2_cat = data_v2.drop(['price', 'year', 'odometer', 'lat', 'long'], axis = 1)
data_v2_cat

,region,manufacturer,fuel,title_status,transmission,type,state
0,prescott,NaN,NaN,NaN,NaN,NaN,az
1,fayetteville,NaN,NaN,NaN,NaN,NaN,ar
2,florida keys,NaN,NaN,NaN,NaN,NaN,fl
3,worcester / central MA,NaN,NaN,NaN,NaN,NaN,ma
4,greensboro,NaN,NaN,NaN,NaN,NaN,nc
...,...,...,...,...,...,...,...
426875,wyoming,nissan,gas,clean,other,sedan,wy
426876,wyoming,volvo,gas,clean,other,sedan,wy
426877,wyoming,cadillac,diesel,clean,other,hatchback,wy
426878,wyoming,lexus,gas,clean,other,sedan,wy


In [19]:
pd_d

,price,year,odometer,lat,long
0,6000.0,2011.235191,98043.331443,38.493940,-94.748599
1,11900.0,2011.235191,98043.331443,38.493940,-94.748599
2,21000.0,2011.235191,98043.331443,38.493940,-94.748599
3,1500.0,2011.235191,98043.331443,38.493940,-94.748599
4,4900.0,2011.235191,98043.331443,38.493940,-94.748599
...,...,...,...,...,...
426875,23590.0,2019.000000,32226.000000,33.786500,-84.445400
426876,30590.0,2020.000000,12029.000000,33.786500,-84.445400
426877,34990.0,2020.000000,4174.000000,33.779214,-84.411811
426878,28990.0,2018.000000,30112.000000,33.786500,-84.445400


In [20]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, attribute_names):
    self.attribute_names = attribute_names
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X[self.attribute_names].values

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [22]:
num_attribs = list(data_v2_num)
cat_attribs = ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state']

num_pipeline = Pipeline([
              ('selector', DataFrameSelector(num_attribs)),
              ('impute', SimpleImputer(strategy = 'mean')),
              ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
              ('selector', DataFrameSelector(cat_attribs)),
              ('cat_encoder', OneHotEncoder(categories=cat_attribs)),
])

In [23]:
from sklearn.pipeline import FeatureUnion

In [24]:
full_pipeline = FeatureUnion(transformer_list=[
                                               ('num_pipeline', num_pipeline),
                                               ('cat_pipeline', cat_pipeline),

                                               
])

In [25]:
# dataset_prepared = full_pipeline.fit_transform(data_v2)

In [27]:
def onehot_encode(df, columns, prefixes):
  df = df.copy()
  for column, prefix in zip(columns, prefixes):
    dummies = pd.get_dummies(df[column], prefix = prefix)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
  return df

In [28]:
data_v2_1h = onehot_encode(data_v2, ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state'], ['reg', 'manu', 'fuel', 'title', 'trans', 'type', 'state'])

In [29]:
data_v2_1h

,price,year,odometer,lat,long,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,reg_altoona-johnstown,reg_amarillo,reg_ames,reg_anchorage / mat-su,reg_ann arbor,reg_annapolis,reg_appleton-oshkosh-FDL,reg_asheville,reg_ashtabula,reg_athens,reg_atlanta,reg_auburn,reg_augusta,reg_austin,reg_bakersfield,reg_baltimore,reg_baton rouge,reg_battle creek,reg_beaumont / port arthur,reg_bellingham,reg_bemidji,reg_bend,reg_billings,reg_binghamton,reg_birmingham,reg_bismarck,reg_bloomington,reg_bloomington-normal,reg_boise,reg_boone,...,state_hi,state_ia,state_id,state_il,state_in,state_ks,state_ky,state_la,state_ma,state_md,state_me,state_mi,state_mn,state_mo,state_ms,state_mt,state_nc,state_nd,state_ne,state_nh,state_nj,state_nm,state_nv,state_ny,state_oh,state_ok,state_or,state_pa,state_ri,state_sc,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
0,6000,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,11900,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,21000,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1500,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4900,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,2019.0,32226.0,33.786500,-84.445400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
426876,30590,2020.0,12029.0,33.786500,-84.445400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
426877,34990,2020.0,4174.0,33.779214,-84.411811,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
426878,28990,2018.0,30112.0,33.786500,-84.445400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [30]:
data_v2_1h["reg_SF bay area"]

0         0
1         0
2         0
3         0
4         0
         ..
426875    0
426876    0
426877    0
426878    0
426879    0
Name: reg_SF bay area, Length: 426880, dtype: uint8

In [31]:
#Removing NaNs
for column in data_v2_1h.columns:
  data_v2_1h[column] = data_v2_1h[column].fillna(data_v2_1h[column].mean())

In [32]:
data_v2_1h.isna().sum().sum()

0

### Splitting and Scaling

In [33]:
y = data_v2_1h.loc[:, 'price']
X = data_v2_1h.drop('price', axis=1)

In [34]:
scaler = StandardScaler()

In [35]:
X = scaler.fit_transform(X)

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 42)

### Training

In [38]:
from sklearn.linear_model import LinearRegression

In [39]:
linear_model = LinearRegression()

In [41]:
linear_model.fit(X_train, y_train)

lin_y_preds = linear_model.predict(X_test)


### Using lightGBM for gradient-boosting

In [43]:
import lightgbm as lgb  

In [44]:
lgb_model = lgb.LGBMRegressor(
    boosting_type = 'gbdt',
    reg_lambda = 1.0
)
lgb_model.fit(X_train, y_train)

lgb_y_preds = lgb_model.predict(X_test)

In [47]:
from sklearn.metrics import mean_squared_error

In [48]:
lin_loss = np.sqrt(mean_squared_error(y_test, lin_y_preds))
lgb_loss = np.sqrt(mean_squared_error(y_test, lgb_y_preds))

In [49]:
print('Linear Regression RMSE: ', lin_loss)
print('LGB RMSE: ', lgb_loss)

Linear Regression RMSE:  1.0742660579917032e+17
LGB RMSE:  16263542.54531327


In [51]:
print('Linear Regression R^2 Score: ', linear_model.score(X_test, y_test))
print('GBM R^2 Score: ', lgb_model.score(X_test, y_test))

#Can be better!

Linear Regression R^2 Score:  -4.4219872234120225e+19
GBM R^2 Score:  -0.013500752131093918


#Optimization